## Web Scrapping del contenido de las url's

A partir del dataframe obtenido en el notebook pasado (notebook del ejercicio), agregamos el texto contenido en los pdf a los que los url's nos dirige.

In [97]:
# Agregamos las bibliotecas necesarias
import re
import string
import locale
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from requests_html import HTMLSession
from collections import defaultdict
from unidecode import unidecode
from datetime import datetime
import io
from PyPDF2 import PdfFileReader # Biblioteca necesaria para leer el contenido del pdf y tratarlo como texto
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Leemos el dataframe con la información necesaria
df = pd.read_pickle("fernando_garcia.pkl")

In [53]:
df.head(5)

,text,url,contenido
date,,,
2008-02-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,15 de Febrero de 2008 Comunicado de Prensa ...
2008-03-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,NaN
2008-04-18,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,NaN
2008-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,NaN
2008-06-20,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,NaN


In [5]:
df.tail(5)

,text,url
date,,
2019-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...
2019-06-27,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...
2019-08-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...
2019-09-26,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...
2019-11-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...


In [67]:
# Agregamos como texto el contenido de los pdf a los que nos dirigen las url's y después agregamos ese texto al dataframe.

for i in df.index:
    with HTMLSession() as sess:
        r = sess.get(df.loc[i,"url"])
        f = io.BytesIO(r.content)
        reader = PdfFileReader(f)
        contents = reader.getPage(0).extractText()
        df.loc[i,"contenido"] = contents

In [72]:
df.head()

,text,url,contenido
date,,,
2008-02-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,15 de Febrero de 2008 Comunicado de Prensa ...
2008-03-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,14 de Marzo de 2008 Comunicado de Prensa An...
2008-04-18,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,18 de Abril de 2008 \nComunicado de Prensa Anu...
2008-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,16 de mayo de 2008 Comunicado de Prensa Anunc...
2008-06-20,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,20 de junio de 2008 Comunicado de Prensa Anun...


In [73]:
df.tail()

,text,url,contenido
date,,,
2019-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,1 16 de \nmayo\n de 201\n9 Comunicado de Pr...
2019-06-27,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,1 27 de \njunio\n de 201\n9 Comunicado de P...
2019-08-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,1 15 de agosto de 2019\n Comunicado de Pren...
2019-09-26,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,1 26 de \nseptiembre\n de 2019\n Comunicado...
2019-11-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,1 14 de \nnoviembre\n de 201\n9 Comunicado d...


In [70]:
df.shape

(104, 3)

Guardamos nuevamente el dataframe, ahora con el corpus de cada entrada url que se tiene en cada renglón guardado dentro de la columna contenido.

In [71]:
df.to_pickle("df_plus_contenido.pkl")

## Limpieza del corpus

In [106]:
# Aplicamos una primera fase de limpieza para eliminar puntuación, saltos de línea y hacer minúsculas al texto
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [107]:
# Echamos un vistazo al dataframe con la primer limpieza realizada
data_clean = pd.DataFrame(df.contenido.apply(round1))
data_clean

,contenido
date,
2008-02-15,de febrero de comunicado de prensa anunc...
2008-03-14,de marzo de comunicado de prensa anuncio ...
2008-04-18,de abril de \ncomunicado de prensa anuncio d...
2008-05-16,de mayo de comunicado de prensa anuncio de ...
2008-06-20,de junio de comunicado de prensa anuncio de...
...,...
2019-05-16,de \nmayo\n de \n comunicado de prensa \n...
2019-06-27,de \njunio\n de \n comunicado de prensa \...
2019-08-15,de agosto de \n comunicado de prensa \n a...


In [108]:
data_clean['contenido'][0]

'  de febrero de  comunicado de prensa    anuncio de política monetaria  la junta de gobierno del banco de méxico ha\n decidido mantener sin cambio el objetivo \npara la tasa de interés interbancaria a  día en  por ciento \n las perspectivas para el crecimiento de la ec\nonomía mundial han continuado deteriorándose  la \nactividad económica en japón y en europa se ha debilitado  la desaceleración ha sido más \n\npronunciada en estados unidos y se\n han acrecentado los riesgos de \nque ocurra una recesión en la \nprimera mitad del año en curso  ello como cons\necuencia de la crisis del mercado hipotecario y \nsus repercusiones en el sector financiero en \notros segmentos del mercado crediticio y en la \nconfianza de los consumidores  el dinamismo \ndel resto de las economías sería afectado por una \ndesaceleración estadounidense acentuada  al\n mismo tiempo han persistido las presiones \ninflacionarias derivadas del alza de\n los precios internacionales de los alimentos destaca el alza 

In [109]:
# Aplicamos una segunda fase de limpieza para quedarnos solo con las palabras que son importantes
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [110]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.contenido.apply(round2))
data_clean

,contenido
date,
2008-02-15,de febrero de comunicado de prensa anunc...
2008-03-14,de marzo de comunicado de prensa anuncio ...
2008-04-18,de abril de comunicado de prensa anuncio de ...
2008-05-16,de mayo de comunicado de prensa anuncio de ...
2008-06-20,de junio de comunicado de prensa anuncio de...
...,...
2019-05-16,de mayo de comunicado de prensa anuncio...
2019-06-27,de junio de comunicado de prensa anunci...
2019-08-15,de agosto de comunicado de prensa anunc...


In [111]:
data_clean['contenido'][0]

'  de febrero de  comunicado de prensa    anuncio de política monetaria  la junta de gobierno del banco de méxico ha decidido mantener sin cambio el objetivo para la tasa de interés interbancaria a  día en  por ciento  las perspectivas para el crecimiento de la economía mundial han continuado deteriorándose  la actividad económica en japón y en europa se ha debilitado  la desaceleración ha sido más pronunciada en estados unidos y se han acrecentado los riesgos de que ocurra una recesión en la primera mitad del año en curso  ello como consecuencia de la crisis del mercado hipotecario y sus repercusiones en el sector financiero en otros segmentos del mercado crediticio y en la confianza de los consumidores  el dinamismo del resto de las economías sería afectado por una desaceleración estadounidense acentuada  al mismo tiempo han persistido las presiones inflacionarias derivadas del alza de los precios internacionales de los alimentos destaca el alza reciente del trigo y de los energético

Finalmente, creamos un nuevo dataframe que contiene ahora a la data_clean junto con los datos anteriores del dataframe original y lo guardamos para usarlo más tarde.

In [112]:
df_1 = df[["text", "url"]]
df_1["contenido"] = data_clean
df_1.to_pickle("df_corpus.pkl")
df_1.head()

,text,url,contenido
date,,,
2008-02-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de febrero de comunicado de prensa anunc...
2008-03-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de marzo de comunicado de prensa anuncio ...
2008-04-18,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de abril de comunicado de prensa anuncio de ...
2008-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de mayo de comunicado de prensa anuncio de ...
2008-06-20,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de junio de comunicado de prensa anuncio de...


## Tokenización

The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [117]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words

cv = CountVectorizer()
data_cv = cv.fit_transform(df_1.contenido)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,abastecimiento,abasto,abatimiento,abatir,abierto,abril,abriljunio,abrió,abrupta,absorber,...,éstos,índice,índices,índole,última,últimas,último,últimos,ﬁancladasﬂ,ﬁanclajeﬂ
date,,,,,,,,,,,,,,,,,,,,,
2008-02-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2008-03-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2008-04-18,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2008-05-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2008-06-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-16,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2019-06-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2019-08-15,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
#data_dtm.columns.tolist()